# 예상 갱신보험료 자동화

#### 모듈 불러오기

- RenewalPremium : RPA용으로 만든 모듈
- pandas : 예상갱신 보험료를 데이터 프레임 형태로 다루기 위한 라이브러리
- tqdm : 반복문을 돌릴 때 진척사항을 보기 위한 라이브러리

In [1]:
from rpa_renewal_pemium import ProductModel, RenewalPremium
import pandas as pd
from tqdm import tqdm
import os

C:\Users\N\AppData\Local\Temp\ipykernel_10508\465278554.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


#### 설정값

- RENEWAL_PREMIUM_LIST_PATH : 예상 갱신보험료 파일 경로
- TEMP_DIR : HTML 파일을 저장할 폴더 경로
- OUTPUT_DIR : PDF 파일을 저장할 폴더 경로

In [2]:
RENEWAL_PREMIUM_LIST_PATH = './input/renewal_premium.txt'
TEMP_DIR = './output/temp'
OUTPUT_DIR = './output'

In [3]:
rpa = RenewalPremium()      # RPA 인스턴스
os.makedirs(os.path.abspath(TEMP_DIR), exist_ok=True)
os.makedirs(os.path.abspath(OUTPUT_DIR), exist_ok=True)

#### 예상 갱신보험료 정보 읽어오기

In [4]:
df_renewal = pd.read_csv(RENEWAL_PREMIUM_LIST_PATH, sep='\t')
print(f"{df_renewal.shape[0]}개의 행을 불러옴")
df_renewal.head()

11개의 행을 불러옴


,상품명,급여명,연령,보험료,기준
0,(무) 메리츠 The좋은 케어프리보험 M-Basket2401(1종)(1형),갱신형 뇌출혈진단비,60,3740,"남자, 40세, 상해1급"
1,(무) 메리츠 The좋은 케어프리보험 M-Basket2401(1종)(1형),갱신형 일반상해80%이상후유장해,60,840,"남자, 40세, 상해1급"
2,(무) 메리츠 The좋은 케어프리보험 M-Basket2401(1종)(1형),갱신형 일반상해사망,60,8300,"남자, 40세, 상해1급"
3,(무) 메리츠 The좋은 케어프리보험 M-Basket2401(1종)(1형),갱신형 질병80%이상후유장해,60,3594,"남자, 40세, 상해1급"
4,(무) 메리츠 The좋은 케어프리보험 M-Basket2401(1종)(1형),갱신형 질병사망,60,23240,"남자, 40세, 상해1급"


#### 예상 갱신보험료 업로드용 파일 생성하기

- 상품별로 예상갱신보험료 파일을 HTML 형태 저장
- HTML을 PDF로 변환하기

In [5]:
for (product_name, df) in tqdm(df_renewal.groupby(['상품명'])):

    product_name = product_name[0]

    try:                                    
    
        # 상품정보 세팅하기
        product = ProductModel(product_name=product_name)
        for row in df.values:
            _, coverage_name, age, premium, condition = row
            product.update(
                coverage_name=coverage_name,
                age=age,
                premium=premium,
                condition=condition
            )
        rpa.set_product(product=product)
 
        # HTML 문서 생성
        rpa.save_html(TEMP_DIR)

        # HTML → PDF로 변환
        rpa.html_to_pdf(html_dir=TEMP_DIR, pdf_dir=OUTPUT_DIR, product_name=product_name)

    # 오류 처리
    except:
        print(f"[ERROR] {product_name} 작업 中 오류가 발생했습니다")

100%|██████████| 1/1 [00:11<00:00, 11.71s/it]
